# 1. Download and prepare RGB stars catalogue

In [1]:
import pathlib

import numpy as np
import pandas as pd

import astropy
import astropy.units as u
from astropy.coordinates import SkyCoord, Galactocentric

import config
import utils

hpx_order=7 --> (hpx_nside=128, hpx_npix=196608)
model_hpx_order=5 --> (model_hpx_nside=32, model_hpx_npix=12288)


In [2]:
cache_path = pathlib.Path(config.cache_path)
cache_path.mkdir(exist_ok=True)

fig_path = pathlib.Path(config.fig_path)
fig_path.mkdir(exist_ok=True)

## Download Andrae et al. (2023) vetted sample of RGB stars

Paper: https://ui.adsabs.harvard.edu/abs/2023ApJS..267....8A

Data: https://zenodo.org/records/7945154

In [3]:
# Download the Andrae et al. (2023) vetted sample of RGB stars

url = 'https://zenodo.org/records/7945154/files/table_2_catwise.csv.gz?download=1'
file_name = cache_path / 'Andrae2023_table_2_catwise.csv.gz'
utils.download_file(url, file_name)

Starting download from https://zenodo.org/records/7945154/files/table_2_catwise.csv.gz?download=1
File 'cache/Andrae2023_table_2_catwise.csv.gz' already downloaded
Done


In [4]:
# Read in the downloaded catalogue
# Use only necessary columns
usecols = ['source_id', 'ra', 'dec', 'l', 'b', 'parallax', 'parallax_error', \
           'pmra', 'pmdec', 'radial_velocity', \
           'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', \
           'mh_xgboost', 'teff_xgboost', 'logg_xgboost']
rgb = pd.read_csv(file_name, usecols=usecols)
print("Num. of RGB stars:", len(rgb))

Num. of RGB stars: 17558141


## Make the transformation from Galactic to Galactocentric frame

In [5]:
# Use inverse parallax as a distance measure
rgb['dist'] = 1 / rgb['parallax']

In [6]:
# The Galactocentric frame
gc_frame = Galactocentric()

# Use the default position and velocity of the Sun
# At the September 2024, the coordinates are taken from GRAVITY Collaboration et al. (2018) and Bennett & Bovy (2019);
# velocities are taken brom Drimmel & Poggio (2018), GRAVITY Collaboration et al. (2018), Reid & Brunthaler (2004)
c_sun_icrs = SkyCoord(0*u.deg, 0*u.deg, 0*u.kpc, pm_ra_cosdec=0*u.mas/u.yr, pm_dec=0*u.mas/u.yr, radial_velocity=0*u.km/u.s, frame='icrs')
c_sun_gc = c_sun_icrs.transform_to(gc_frame)

x_sun = c_sun_gc.x.value
y_sun = c_sun_gc.y.value
z_sun = c_sun_gc.z.value
print("Sun Cartesian pos:", x_sun, y_sun, z_sun)

vx_sun = c_sun_gc.v_x.value
vy_sun = c_sun_gc.v_y.value
vz_sun = c_sun_gc.v_z.value
print("Sun Cartesian vel:", vx_sun, vy_sun, vz_sun)

Sun Cartesian pos: -8.1219733661223 0.0 0.020800000000000003
Sun Cartesian vel: 12.899999999999999 245.6 7.779999999999999


In [7]:
# Clickable references:
astropy.coordinates.galactocentric_frame_defaults.get_from_registry('latest')

{'parameters': {'galcen_coord': <ICRS Coordinate: (ra, dec) in deg
      (266.4051, -28.936175)>,
  'galcen_distance': <Quantity 8.122 kpc>,
  'galcen_v_sun': <CartesianDifferential (d_x, d_y, d_z) in km / s
      (12.9, 245.6, 7.78)>,
  'z_sun': <Quantity 20.8 pc>,
  'roll': <Quantity 0. deg>},
 'references': {'galcen_coord': 'https://ui.adsabs.harvard.edu/abs/2004ApJ...616..872R',
  'galcen_distance': 'https://ui.adsabs.harvard.edu/abs/2018A%26A...615L..15G',
  'galcen_v_sun': ['https://ui.adsabs.harvard.edu/abs/2018RNAAS...2..210D',
   'https://ui.adsabs.harvard.edu/abs/2018A%26A...615L..15G',
   'https://ui.adsabs.harvard.edu/abs/2004ApJ...616..872R'],
  'z_sun': 'https://ui.adsabs.harvard.edu/abs/2019MNRAS.482.1417B',
  'roll': None}}

In [8]:
# Another way to define the Galactic frame when we like someone's estimate for the Sun position and velocity
#gc_frame = Galactocentric(galcen_distance=abs(x_sun)*u.kpc, z_sun=z_sun*u.kpc, galcen_v_sun=[vx_sun, vy_sun, vz_sun]*u.km/u.s)

In [9]:
# Transfrom
# Actually we don't need velocities for our model but let them be

c_icrs = SkyCoord(rgb['ra'].values * u.deg,
                  rgb['dec'].values * u.deg,
                  rgb['dist'].values * u.kpc,
                  pm_ra_cosdec=rgb['pmra'].values * u.mas/u.yr,
                  pm_dec=rgb['pmdec'].values * u.mas/u.yr,
                  radial_velocity=rgb['radial_velocity'].values * u.km/u.s,
                  frame='icrs')

c_gc = c_icrs.transform_to(gc_frame)

x = c_gc.x.to(u.kpc).value
y = c_gc.y.to(u.kpc).value
z = c_gc.z.to(u.kpc).value

vx = c_gc.v_x.to(u.km/u.s).value
vy = c_gc.v_y.to(u.km/u.s).value
vz = c_gc.v_z.to(u.km/u.s).value

rgb['x'], rgb['y'], rgb['z']    =  x,  y,  z
rgb['vx'], rgb['vy'], rgb['vz'] = vx, vy, vz

## Save RGB stars catalogue

In [11]:
# We don't care of NaNs in velocities or extinctions
print("NaNs:")
print(rgb.isna().sum())

print("Columns:", rgb.columns)

# The HDF5 format is fast!
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_hdf.html#pandas.DataFrame.to_hdf
# https://pandas.pydata.org/docs/reference/api/pandas.read_hdf.html#pandas.read_hdf
columns = ['source_id', 'l', 'b', 'parallax', 'parallax_error', 'dist', 'x', 'y', 'z', 'vx', 'vy', 'vz', \
           'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', \
           'teff_xgboost', 'mh_xgboost']
data_columns = ['source_id', 'l', 'b', 'parallax', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'teff_xgboost', 'mh_xgboost']
complevel = 0  # 0..9
rgb[columns].to_hdf(cache_path / 'rgb.hdf5', key='rgb', format='table', data_columns=data_columns, complevel=complevel, mode='w', index=False)

NaNs:
source_id                 0
l                         0
b                         0
ra                        0
dec                       0
parallax                  0
parallax_error            0
pmra                      0
pmdec                     0
radial_velocity     4982977
phot_g_mean_mag           0
phot_bp_mean_mag          0
phot_rp_mean_mag          0
mh_xgboost                0
teff_xgboost              0
logg_xgboost              0
dist                      0
x                         0
y                         0
z                         0
vx                  4982977
vy                  4982977
vz                  4982977
dtype: int64
Columns: Index(['source_id', 'l', 'b', 'ra', 'dec', 'parallax', 'parallax_error',
       'pmra', 'pmdec', 'radial_velocity', 'phot_g_mean_mag',
       'phot_bp_mean_mag', 'phot_rp_mean_mag', 'mh_xgboost', 'teff_xgboost',
       'logg_xgboost', 'dist', 'x', 'y', 'z', 'vx', 'vy', 'vz'],
      dtype='object')
